In [2]:
# Memanggil library yang dibutuhkan
import ee
import geemap

# Autentikasi dan Inisialisasi
ee.Authenticate()
ee.Initialize()

# Membuat peta interaktif
Map = geemap.Map()

# --- Path ke ASET BARU Anda untuk tahun 2016 yang sudah ditambal ---
path_aset_lc_2016 = "users/ivan_taslim/LC_Paguyaman_2016_DW_Filled"

# Memuat raster tutupan lahan 2016 yang baru
lc_2016 = ee.Image(path_aset_lc_2016)
aoi = lc_2016.geometry()

# Menampilkan AOI di peta
Map.addLayer(aoi, {'color': 'yellow'}, 'Batas DAS Paguyaman (AOI)')
Map.centerObject(aoi, 11)
Map

Map(center=[0.7139019338201237, 122.37991662440572], controls=(WidgetControl(options=['position', 'transparent…

In [3]:
# Berdasarkan skema reklasifikasi baru, kelas pertanian adalah:
# 3: Sawah / Lahan basah
# 4: Pertanian / Perkebunan

# Membuat gambar biner (nilai 1 untuk area pertanian, 0 untuk lainnya)
agriculture_mask = lc_2016.eq(3).Or(lc_2016.eq(4))
agriculture_mask = agriculture_mask.updateMask(agriculture_mask.eq(1))

# Menampilkan area pertanian yang akan dianalisis
Map.addLayer(agriculture_mask, {'palette': ['#E5A429']}, 'Masker Lahan Pertanian 2016')

print("Masker untuk memfilter area pertanian 2016 telah dibuat.")

Masker untuk memfilter area pertanian 2016 telah dibuat.


In [4]:
# Fungsi untuk menghilangkan awan dari citra Landsat
def mask_lansat_clouds(image):
    qa_pixel = image.select('QA_PIXEL')
    cloud_shadow, clouds = (1 << 3), (1 << 4)
    mask = qa_pixel.bitwiseAnd(cloud_shadow).eq(0).And(qa_pixel.bitwiseAnd(clouds).eq(0))
    return image.updateMask(mask).divide(10000).copyProperties(image, ["system:time_start"])

# Menghitung NDVI untuk tahun 2016
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
landsat_2016 = landsat_collection.filterBounds(aoi).filterDate('2016-01-01', '2016-12-31')
landsat_2016_no_clouds = landsat_2016.map(mask_lansat_clouds)
median_2016 = landsat_2016_no_clouds.median()
ndvi_2016 = median_2016.normalizedDifference(['SR_B5', 'SR_B4']).rename('ndvi')

# Menampilkan peta NDVI
ndvi_palette = ['#FFFFFF', '#CE7E45', '#DF923D', '#F1B555', '#FCD163', '#99B718', '#74A901', '#66A000', '#529400', '#3E8601', '#207401', '#056201', '#004C00', '#023B01', '#012E01', '#011D01', '#011301']
Map.addLayer(ndvi_2016.clip(aoi), {'min': 0, 'max': 1, 'palette': ndvi_palette}, 'Peta NDVI 2016')

print("Peta NDVI untuk tahun 2016 berhasil dibuat.")

Peta NDVI untuk tahun 2016 berhasil dibuat.


In [5]:
# Menerapkan masker pertanian ke peta NDVI
ndvi_pertanian_2016 = ndvi_2016.updateMask(agriculture_mask)

# Menghitung statistik (rata-rata) NDVI di area pertanian
stats_ndvi_2016 = ndvi_pertanian_2016.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=aoi,
    scale=30,
    maxPixels=1e12
)

# Menampilkan hasil
avg_ndvi_2016 = stats_ndvi_2016.get('ndvi').getInfo()
print("======================================================================")
print("HASIL ANALISIS ULANG PRODUKTIVITAS LAHAN (NDVI) TAHUN 2016")
print("----------------------------------------------------------------------")
print(f"NDVI Rata-rata di Lahan Pertanian: {avg_ndvi_2016:.4f}")
print("======================================================================")

HASIL ANALISIS ULANG PRODUKTIVITAS LAHAN (NDVI) TAHUN 2016
----------------------------------------------------------------------
NDVI Rata-rata di Lahan Pertanian: 0.2920


In [6]:
# ======================================================================
# LANGKAH 5: EKSPOR PETA NDVI 2016
# ======================================================================

# 5.1 Ekspor ke GEE Assets
task_asset = ee.batch.Export.image.toAsset(
  image=ndvi_2016.toFloat(), # Konversi ke float untuk ekspor
  description='NDVI_Paguyaman_2016_to_Asset',
  assetId='users/ivan_taslim/NDVI_Paguyaman_2016', # Path aset tujuan Anda
  region=aoi,
  scale=30,
  maxPixels=1e12
)
task_asset.start()
print("Proses ekspor ke GEE ASSETS telah dimulai.")

# 5.2 Ekspor ke Google Drive
task_drive = ee.batch.Export.image.toDrive(
  image=ndvi_2016.toFloat(),
  description='NDVI_Paguyaman_2016_to_Drive',
  folder='Hasil_Analisis_DAS_Paguyaman',
  fileNamePrefix='NDVI_Paguyaman_2016',
  region=aoi,
  scale=30,
  maxPixels=1e12
)
task_drive.start()
print("Proses ekspor ke GOOGLE DRIVE telah dimulai.")
print("\nPERIKSA TAB 'TASKS' DI GEE CODE EDITOR UNTUK MENJALANKAN KEDUA PROSES EKSPOR.")

Proses ekspor ke GEE ASSETS telah dimulai.
Proses ekspor ke GOOGLE DRIVE telah dimulai.

PERIKSA TAB 'TASKS' DI GEE CODE EDITOR UNTUK MENJALANKAN KEDUA PROSES EKSPOR.
